<a href="https://colab.research.google.com/github/icekang/NLP_2021/blob/main/hw4_language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM (optional)



In [81]:
#download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2021-02-14 09:43:57--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2021-02-14 09:43:58--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip.1’

BEST2010.zip.1      100%[===================>]   7.08M  42.5MB/s    in 0.2s    

2021-02-14 09:43:58 (42.5 MB/s) - ‘BEST2010.zip.1’ saved [7423530/7423530]

Archive:  BEST2010.zip
replace BEST2010/artic

In [82]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [117]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [118]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [119]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [86]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count = 0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [87]:
model = getUnigramModel(train)

In [88]:
def getLnValue(x):
    if x >0.0:
      return math.log(x)
    else:
      return math.log(unk_value)

In [89]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

# TODO #0
**Fill your name and ID here** <br>
**Name**: Naravich Chutisilp
<br>
**ID**: 6030319921

## TODO #1 **Calculate perplexity**

In [90]:
def calculate_sentence_ln_prob(tokenized_sentence, model):
  _sum = 0
  for word in tokenized_sentence:
     _sum += getLnValue(model[word])
  
  return _sum

def perplexity(test, model):
  _sum = 0
  _cnt = 0
  for sentence in test:
    sentence += u'|</s>'
    tokenized_sentence = sentence.split('|')
    _sum += calculate_sentence_ln_prob(tokenized_sentence, model)
    _cnt += len(tokenized_sentence)
  return math.exp(-1 * _sum / _cnt)

In [91]:
print(perplexity(train,model))
print(perplexity(test,model))
# 556.3925994212195
# 476.0687892303532

556.3925994212195
476.0687892303532


## TODO #2 **Please explain why this model give us such a high perplexity.**

**Your answer**: the model does not consider words before them. Like us, only considering which word should come after another gives quite wrong answer.

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [92]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.') # อยากลืม <s> กับ </s>; normalize ด้วย 9
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.54 and 145.88 on train and test data.

## TODO #3 **Create a Bigram Model**

In [93]:
def getBigramModel(data):
  ###FILL YOUR CODE HERE###
  model = defaultdict(lambda: 0.)
  bigram_count = defaultdict(lambda: 0.)
  unigram_count = defaultdict(lambda: 0.)
  _cnt = 0
  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      bigram_count[(w1,w2)] += 1.
      unigram_count[w2] += 1.
    _cnt += 1
    if _cnt % 1000 == 0:
      print('_cnt', _cnt)

  print('getting model', len(bigram_count.keys()))
  for w1, w2 in bigram_count.keys():
    model[(w1,w2)] = bigram_count[(w1,w2)] / unigram_count[w1]

  return model

model = getBigramModel(train)

_cnt 1000
_cnt 2000
_cnt 3000
_cnt 4000
_cnt 5000
_cnt 6000
_cnt 7000
_cnt 8000
_cnt 9000
_cnt 10000
_cnt 11000
_cnt 12000
_cnt 13000
_cnt 14000
_cnt 15000
_cnt 16000
_cnt 17000
_cnt 18000
_cnt 19000
_cnt 20000
_cnt 21000
getting model 224393


## TODO #4 **Calculate Perplexity for Bigram Model**



In [94]:
def calculate_sentence_ln_prob(sentence, model):
  _sum = 0
  _n = 0
  for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
    _sum += getLnValue(model[(w1, w2)])
    _n += 1
  return _sum, _n

def perplexity(test,model):
  _sum = 0
  _cnt = 0
  for sentence in test:
    s, n = calculate_sentence_ln_prob(sentence, model)
    _sum += s
    _cnt += n
  return math.exp(-1 * _sum / _cnt)

In [95]:
print(perplexity(train,model))
print(perplexity(test, model))

# 58.78942889767147
# 146.26539331038614

58.78942889767147
146.26539331038614


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #5 **Bigram with Laplace smoothing (Add-One Smoothing)**

In [96]:
#Laplace Smoothing

# for functional model
# def calculate_sentence_ln_prob(sentence, model):
#   _sum = 0
#   _n = 0
#   for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
#     _sum += getLnValue(model(w1, w2))
#     _n += 1
#   return _sum, _n

def getBigramWithLaplaceSmoothing(data):
  bigram_count = defaultdict(lambda: 0.)
  unigram_count = defaultdict(lambda: 0.)
  model = defaultdict(lambda: 0.)
  _cnt = 0
  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      bigram_count[(w1,w2)] += 1.
      unigram_count[w2] += 1.
    _cnt += 1

  V = len(unigram_count.keys())
  for w1, w2 in bigram_count.keys():
    model[(w1, w2)] = (bigram_count[(w1,w2)] + 1) / (unigram_count[w1] + V)
  
  # def model(w1, w2):
  #   if (w1,w2) not in bigram_count:
  #     return math.pow(len(train_vocabulary),-1)
  #   return (bigram_count[(w1,w2)] + 1) / (unigram_count[w1] + V)

  return model

model = getBigramWithLaplaceSmoothing(train)
print (perplexity(train, model))
print (perplexity(test, model))

# 974.8134581679766
# 1098.1622194979489

974.8134581679766
1098.1622194979489


## TODO #6 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [100]:
#interpolation
def getUnigramModel2(data,word_count):
  model = defaultdict(lambda: 0)
  for sentence in data:
    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      model[w1] +=1.0
  for w1 in model:
    model[w1] = model[w1]/word_count
  return model

def getBigramWithInterpolation(data):
  bigram_count = defaultdict(lambda: 0.)
  unigram_count = defaultdict(lambda: 0.)
  unigram_model = getUnigramModel2(data, train_word_count)
  model = defaultdict(lambda: 0.)
  _cnt = 0
  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      bigram_count[(w1,w2)] += 1.
      unigram_count[w2] += 1.
    _cnt += 1

  V = len(unigram_count.keys())
  for w1, w2 in bigram_count.keys():
    b = bigram_count[(w1,w2)] / unigram_count[w1]
    u = unigram_model[w2]
    C = unk_value
    model[(w1, w2)] = 0.7*b + 0.25*u + 0.05*C

  return model
    
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))

# 73.38409869825665
# 172.67485908813356

73.38409869825665
172.67485908813356


the result perplexity on training and testing should be 

    966.26, 1092.15 for Laplace smoothing
    73.01, 170.97 for Interpolation

# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [101]:
# create article data
encyclo_data=[]
fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()


First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "ariticle data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.79.

In [102]:
#print perplexity of bigram with interpolation on article data        
print (perplexity(encyclo_data,model))

727.3502637212223


## TODO #7 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [116]:
# Fill code here
combined_data=[]
with io.open('BEST2010/news.txt','r',encoding='utf-8') as fp:
  for i,line in enumerate(fp):
      combined_data.append(line.strip()[:-1])

sentences = combined_data
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

news_model = getBigramWithInterpolation(combined_data)
news_weight = perplexity(test, news_model)


combined_data=[]
with io.open('BEST2010/article.txt','r',encoding='utf-8') as fp:
  for i,line in enumerate(fp):
      combined_data.append(line.strip()[:-1])

sentences = combined_data
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

article_weight = perplexity(test, article_model)

# Combine model with their perplexity on their test data
total_weight = news_weight + article_weight
bigrams_combinations = set(list(news_model.keys()) + list(article_model.keys()))
combined_model = defaultdict(lambda: 0.)
for w1, w2 in bigrams_combinations:
  combined_model[(w1,w2)] = news_model[(w1,w2)] * news_weight/total_weight + article_model[(w1,w2)] * article_weight/total_weight

print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model))

Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 397.72729147277704


## TODO #8 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 55.06 and 140.76 on train and test data.


In [120]:
# Fill codehere
def kneser_ney_model(data):
  # Probability of Kneser-ney Pkn
  # Bigram, but cooler
  bigram_count = defaultdict(lambda: 0.)
  unigram_count = defaultdict(lambda: 0.)
  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      bigram_count[(w1,w2)] += 1.
      unigram_count[w2] += 1.
  d = 0.75
  bigram_but_cooler = defaultdict(lambda: 0.)
  for w1, w2 in bigram_count.keys():
    bigram_but_cooler[(w1,w2)] = max(bigram_count[(w1,w2)] - d, 0) / unigram_count[w1]

  # P_continuation w(i)
  distinct_bigram_count = len(bigram_count.keys())
  P_continuation = defaultdict(lambda: set())
  for w1, w2 in bigram_count.keys():
    P_continuation[w2].add(w1)
  for w in P_continuation.keys():
    P_continuation[w] = len(P_continuation[w]) / distinct_bigram_count

  # Lambda w(i-1)
  lambda_follow_distinct = defaultdict(lambda: set())
  lambda_follow_non_distinct = defaultdict(lambda: 0.)
  for w1, w2 in bigram_count.keys():
    lambda_follow_distinct[w1].add(w2)
    lambda_follow_non_distinct[w1] += bigram_count[(w1,w2)]
  for w in lambda_follow_distinct.keys():
    lambda_follow_distinct[w] = len(lambda_follow_distinct[w]) / lambda_follow_non_distinct[w]

  model_kn = defaultdict(lambda: 0.)
  for w1, w2 in bigram_count.keys():
    model_kn[(w1,w2)] = bigram_but_cooler[(w1,w2)] + d * lambda_follow_distinct[w1] * P_continuation[w2]
  
  return model_kn

model = kneser_ney_model(train)
print (perplexity(train,model))        
print (perplexity(test,model))
# 71.14054002208687
# 174.02464248000433, 174, 155, 142

71.14054002208687
155.09274968738495


## TODO #9
## Neural LM (optional)
do it on news corpus that we splitted into train and test sets at the beginning of this exercise. 

In [ ]:
#there are many ways to do this. e.g.:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
'''
Embedding -> LSTM -> Dense
'''